In [1]:
import sys
sys.path.insert(1,'/Users/charlesh/Documents/Codes/')
sys.path.append('/Users/charlesh/Documents/Codes/OBS_Methods/NOISE/METHODS')
sys.path.insert(0, '/Users/charlesh/Documents/Codes/OBS_Methods/NOISE/COMPS')
sys.path.insert(0, '/Users/charlesh/Documents/Codes/OBS_Methods/NOISE/METHODS/ATaCR/ATaCR_Python/OBStools')
sys.path.insert(0, '/Users/charlesh/Documents/Codes/OBS_Methods/NOISE/METHODS/ATaCR/ATaCR_Python')

from obstools.atacr import DayNoise, TFNoise, EventStream, StaNoise, utils
import obstools.atacr.plotting as atplot
from pathlib import Path
from obspy.core import read, Stream, Trace, AttribDict, UTCDateTime
import numpy as np
import pandas as pd
import sys
sys.path.insert(0, '/Users/charlesh/Documents/Codes/ATaCR')
import ObsQA
import ObsQA as ob
import datetime
from obspy.clients.fdsn import Client
import glob as g
import os
import argparse
import obspy
from obstools.scripts import comply_calculate, atacr_clean_spectra, atacr_correct_event, atacr_daily_spectra, atacr_download_data, atacr_download_event, atacr_transfer_functions
from stdb.scripts import query_fdsn_stdb

# stations, stations_set = ObsQA.io.getstalist()
NoiseFolder = '/Users/charlesh/Documents/Codes/OBS_Methods/NOISE'
CompFolder = NoiseFolder + '/COMPS/ATaCR_NC'
MethodsFolder = NoiseFolder + '/METHODS/ATaCR'
ATaCR_Py_DataFolder = ob.io.dir_libraries(MethodsFolder)[1]
ATaCR_Parent = ATaCR_Py_DataFolder['Py_DataParentFolder']
eventsfolder = ATaCR_Py_DataFolder['Py_CorrectedTraces']
# eventsfolder = '/Users/charlesh/Documents/Codes/ATaCR/ATaCR_Comp/ATaCR_Python/EVENTS'
catalog = pd.read_pickle(eventsfolder + '/event_catalog.pkl')
# display(catalog)

In [2]:
catalog

,Times,Events,Magnitudes_mw,Origin,Metadata,n_sta
0,2010-09-03T11:16:08.110000Z,2010.246.11.16,6.5,"[resource_id, time, time_errors, longitude, lo...","[resource_id, event_type, event_type_certainty...",36
1,2010-09-03T16:35:46.480000Z,2010.246.16.35,7.0,"[resource_id, time, time_errors, longitude, lo...","[resource_id, event_type, event_type_certainty...",36
2,2010-09-04T08:52:03.690000Z,2010.247.8.52,6.0,"[resource_id, time, time_errors, longitude, lo...","[resource_id, event_type, event_type_certainty...",36
3,2010-09-07T16:13:33.500000Z,2010.250.16.13,6.3,"[resource_id, time, time_errors, longitude, lo...","[resource_id, event_type, event_type_certainty...",36
4,2010-09-08T11:37:37.700000Z,2010.251.11.37,6.3,"[resource_id, time, time_errors, longitude, lo...","[resource_id, event_type, event_type_certainty...",36
5,2010-12-22T21:49:42.030000Z,2010.356.21.49,6.4,"[resource_id, time, time_errors, longitude, lo...","[resource_id, event_type, event_type_certainty...",13
6,2010-12-23T14:00:33.450000Z,2010.357.14.0,6.4,"[resource_id, time, time_errors, longitude, lo...","[resource_id, event_type, event_type_certainty...",13
7,2010-12-24T05:48:54.040000Z,2010.358.5.48,6.0,"[resource_id, time, time_errors, longitude, lo...","[resource_id, event_type, event_type_certainty...",13
8,2010-12-28T08:34:17.100000Z,2010.362.8.34,6.3,"[resource_id, time, time_errors, longitude, lo...","[resource_id, event_type, event_type_certainty...",13
9,2010-12-29T06:54:22.310000Z,2010.363.6.54,6.3,"[resource_id, time, time_errors, longitude, lo...","[resource_id, event_type, event_type_certainty...",13


In [ ]:
# -----------------------------------------------------------------------------------------------------
# QC_Spectra and Clean_Spectra issues 11/13/23:

# 1.  XE.CC05, 7D.M07A, and 7D.M08A each had several days of noise that had to be removed for crashing QC_Spectra.
    # They're quarantened to QC_ISSUES dirs (except for M08As).
    # Possible dead traces. Need to look at these as a possible code vulnerability

# 2. Many stations that had more than ~450 SAC files of noise to run crashed the Python kernel after exceeding ~50gb memory pressure.
#          -Not a bug but likely an ATaCR optimization issue that might need a simple job task manager function to handle batches better.
#          -The memory issue was avoided when breaking the larger jobs up into sections.
#          -SIX stations in total could not finish both processing steps without additional help (they are finished now).
# -----------------------------------------------------------------------------------------------------

In [ ]:
# tf = pd.read_pickle ('/Users/charlesh/Documents/Codes/OBS_Methods/NOISE/METHODS/ATaCR/ATaCR_Python/TF_STA/7D.M08A/2012.140.transfunc.pkl')
# import matplotlib.pyplot as plt
# x,y = tf.f,tf.transfunc['ZP-21']['TF_ZP-21']
# y = y[x>0]
# x = x[x>0]
# y_tf = np.real(y)

# y_mx = y_tf.max()

# plt.plot(x,y_tf,label='tapered',c='k')

# tf_no_taper = y_tf * (1/y_taper)
# tf_no_taper = tf_no_taper * (2/tf_no_taper.max())
# plt.plot(x,tf_no_taper,label='no taper',c='r')
# x,y = tf.f,tf.comp_tf_taper
# y = y[x>0]
# x = x[x>0]
# y_taper = y
# y_taper = y_taper*(y_tf).max()
# plt.plot(x,y_taper,label='comp taper',c='b')
# x = tf.f
# y = tf.tilt_tf_taper
# y = y[x>0]
# x = x[x>0]
# y_taper = y
# y_taper = y_taper*(y_tf).max()
# plt.plot(x,y_taper,label='tilt taper',c='m')
# plt.xscale('log')

# # plt.ylim(-8.409085469715815e-08, 3.0984437734529147e-07)
# yl = plt.gca().get_ylim()

In [ ]:
# os.chdir(ATaCR_Parent)
# staquery_output='./sta_query.pkl'
# extra_flags = '-O'
# args = [staquery_output]
# [args.append(flg) for flg in extra_flags.split()]
# # args
# args = atacr_transfer_functions.get_transfer_arguments(args)
# atacr_transfer_functions.main(args)

In [ ]:
cat = catalog.copy()
STEPS = [1,6,7]

In [ ]:
# ------------------------ATACR Station-by-station and Step-by-step------------------------
# cat = catalog.copy()
# --
# stas = ['FN07A']
# cat = cat[np.in1d(cat.Station,stas)]
# --
# STEPS = [1,6,7]
# --
# --
# ObsQA.io.Run_ATaCR(cat, STEPS=STEPS)
# display(cat)
# Step-1: Station Metadata. Step a0 in ML-ATaCR. Always run this.----|
# -------------------------------------------------------------------|
# Step-2: Download event data. Step a3 in ML-ATaCR.------------------|
# -------------------------------------------------------------------|
# Step-3: Download day data. Step a2 in ML-ATaCR.--------------------|
# -------------------------------------------------------------------|
# Step-4: Daily Spectra. Step b1 in ML-ATaCR.------------------------|
# -------------------------------------------------------------------|
# Step-5: Clean and Average Daily Spectra. Step b2 in ML-ATaCR.------|
# -------------------------------------------------------------------|
# Step-6: Calculate transfer functions. Step b3 ML-ATaCR.------------|
# -------------------------------------------------------------------|
# Step-7: Correct events. Step b4 in ML-ATaCR.-----------------------|
# -------------------------------------------------------------------|
for step in np.array(STEPS)[np.array(STEPS)>1]:
    N = len(cat)
    # N = 1
    for ii in range(N):
        S = cat['Station'].tolist()[ii]
        N = cat['Network'].tolist()[ii]
        staname = N + '.' + S
        subfolder = staname + '/'
        print('Station: ' + staname +  ' (' + str(ii+1) + ' of ' + str(len(cat)) + ')')
        # =============# =============# =============
        icatalog = cat[(cat.Network==N) & (cat.Station==S)].copy()
        ObsQA.io.Run_ATaCR(icatalog,ATaCR_Parent = ATaCR_Parent, STEPS=[1,step])
        print('[*]'*20)
    print('[=*=]'*20)